https://app.roboflow.com/his/uno-cards-yfboa/1

# Импорт проекта из Roboflow

In [1]:
%pip install -q roboflow

     |████████████████████████████████| 42 kB 407 kB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 138 kB 10.1 MB/s 
     |████████████████████████████████| 145 kB 55.0 MB/s 
     |████████████████████████████████| 178 kB 15.0 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 62 kB 952 kB/s 


In [2]:
from roboflow import Roboflow

rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [4]:
rf = Roboflow(api_key="Gq0NZcvklNi9jjGrrjCB")
project = rf.workspace().project("dice-cjrpu")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Dice-1 in yolov5pytorch:: 100%|██████████| 1731/1731 [00:00<00:00, 2180.80it/s]


# Клонирование модели YOLO5

In [5]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 14525, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 14525 (delta 22), reused 29 (delta 14), pack-reused 14477
Receiving objects: 100% (14525/14525), 13.70 MiB | 30.44 MiB/s, done.
Resolving deltas: 100% (9994/9994), done.
/content/yolov5
     |████████████████████████████████| 182 kB 4.4 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 1.6 MB 41.7 MB/s 


In [6]:
!cp -r /content/Dice-1 /content/yolov5/Dice-1

# Дообучение модели


In [7]:
import os
os.environ["DATASET_DIRECTORY"] = "/content/"
!python train.py --img 416 --batch 16 --epochs 100 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/Dice-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-34-g1ae9194 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7

# Распознавание на тестовой выборке

In [19]:
train_n = '' # experiment number

!python detect.py --weights runs/train/exp{train_n}/weights/best.pt --img 416 --save-txt --conf 0.5 --source {dataset.location}/test/images

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/Dice-1/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-34-g1ae9194 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
image 1/36 /content/Dice-1/test/images/IMG_0654_jpeg.rf.96626a2cc059193874751a0724c19c38.jpg: 416x416 1 1, 1 6, 8.1ms
image 2/36 /content/Dice-1/test/images/IMG_0655_jpeg.rf.3694b63bdf09b79b374b04e74f8b3b46.jpg: 416x416 1 1, 1 6, 8.2ms
image 3/36 /content/Dice-1/test/images/IMG_0656_jpeg.rf.caf6987d2f44f93f18b821b08fc9dc4a.jpg: 416x416 1 1

# Формирование датафрейма с результатами распознавания

In [29]:
classes = {0: '1',
           1: '2',
           2: '3',
           3: '4',
           4: '5',
           5: '6'}

In [30]:
import pandas as pd
import os

df = pd.DataFrame(columns=["objects"])

det_n = '3' # experiment number
dir = f'/content/yolov5/runs/detect/exp{det_n}/labels'
for filename in os.listdir(dir):
  with open(os.path.join(dir, filename), 'r') as f:
    text = f.readlines()
    objects = []
    for i in text:
      label = int(i.split(' ')[0])
      objects.append(classes[label])
      index = filename[:-3]+'jpg'
      df.loc[index, 'objects'] = objects

df.head()

,objects
IMG_0753_jpeg.rf.1b520d88c1561ab33146a86214fe38c8.jpg,"[4, 4]"
i.rf.0fb2579fbc8fd9b6af9a2e2ce5ad0b56.jpg,[1]
IMG_0812_jpeg.rf.7669a6540f2a28904c70ff702d3f76a2.jpg,[6]
IMG_0813_jpeg.rf.dc952850e1eb78e512ae04fe473b9c79.jpg,"[1, 6]"
IMG_0796_jpeg.rf.8d145545aadb839edc15cc368c239086.jpg,"[5, 4]"


In [31]:
df.to_csv("/content/result.csv")

# Извлечение отношений

one-hot encoding

In [32]:
def encode(image, objects):
  for x in objects:
    data.loc[image, x] = True

data = pd.DataFrame(index = df.index, columns = classes.values())

for index, row in df.iterrows():
  encode(index, row['objects'])
data = data.fillna(False)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, IMG_0753_jpeg.rf.1b520d88c1561ab33146a86214fe38c8.jpg to IMG_0743_jpeg.rf.c0940f133de7acbab81bc684e295e1ab.jpg
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   1       36 non-null     bool 
 1   2       36 non-null     bool 
 2   3       36 non-null     bool 
 3   4       36 non-null     bool 
 4   5       36 non-null     bool 
 5   6       36 non-null     bool 
dtypes: bool(6)
memory usage: 1.5+ KB


поиск ассоциативных правил

In [33]:
from mlxtend.frequent_patterns import apriori

min_support = 2/len(data.index)
itemsets = apriori(data, min_support=min_support, max_len=2, use_colnames = True)

itemsets.head()

,support,itemsets
0,0.305556,(1)
1,0.250000,(2)
2,0.111111,(3)
3,0.361111,(4)
4,0.277778,(5)


In [34]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(itemsets, min_threshold = min_support)
rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x)[0])
rules['consequents'] = rules['consequents'].apply(lambda x: list(x)[0])

rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,4,1,0.361111,0.305556,0.055556,0.153846,0.503497,-0.054784,0.820707
1,1,4,0.305556,0.361111,0.055556,0.181818,0.503497,-0.054784,0.780864
2,1,6,0.305556,0.444444,0.166667,0.545455,1.227273,0.030864,1.222222
3,6,1,0.444444,0.305556,0.166667,0.375000,1.227273,0.030864,1.111111
4,2,3,0.250000,0.111111,0.055556,0.222222,2.000000,0.027778,1.142857


отбор правил по lift

In [36]:
rels = rules[rules['lift'] > 1]
rels = rels[['antecedents', 'consequents']]

rels.head()

,antecedents,consequents
2,1,6
3,6,1
4,2,3
5,3,2
6,2,4


# Формирование графа знаний

In [37]:
!pip install Cython
!pip install owlready2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.5 MB 92.0 MB/s 
  Created wheel for owlready2: filename=Owlready2-0.39-cp38-cp38-linux_x86_64.whl size=22260891 sha256=3016d0af73c16641cde364e166b6929ed3eb3d3e3360229982c795e89a691506
  Stored in directory: /root/.cache/pip/wheels/bd/16/5f/a0bfc34a8f7682cbd6b4d9cb1436c0a0a04ac3579394d7e28a
Successfully built owlready2


In [38]:
from owlready2 import *

onto = get_ontology('http://onto.owl')

создание экземпляров на основе классов

In [39]:
import types

with onto:
  onto_cl = types.new_class('Dice', (Thing,))
  for i, cl in enumerate(classes.values()):
    inst = onto_cl(f'dice_{i}')
    inst.label = cl

onto.save('/content/onto.owl')

создание транзитивного отношения

In [40]:
with onto:
  types.new_class('isAssociatedWith', (ObjectProperty, TransitiveProperty))
  for index, rel in rels.iterrows():
    onto_ant = onto.search_one(label = rel['antecedents'])
    onto_con = onto.search_one(label = rel['consequents'])
    onto_ant.isAssociatedWith.append(onto_con)

onto.save('/content/onto.owl')

вывод новых отношений

In [41]:
with onto:
  sync_reasoner_pellet(infer_property_values=True)

onto.save('/content/onto_reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-pa

* Owlready * Adding relation onto.dice_4 isAssociatedWith onto.dice_4
* Owlready * Adding relation onto.dice_4 isAssociatedWith onto.dice_0
* Owlready * Adding relation onto.dice_4 isAssociatedWith onto.dice_1
* Owlready * Adding relation onto.dice_0 isAssociatedWith onto.dice_4
* Owlready * Adding relation onto.dice_0 isAssociatedWith onto.dice_0
* Owlready * Adding relation onto.dice_0 isAssociatedWith onto.dice_3
* Owlready * Adding relation onto.dice_0 isAssociatedWith onto.dice_1
* Owlready * Adding relation onto.dice_0 isAssociatedWith onto.dice_2
* Owlready * Adding relation onto.dice_3 isAssociatedWith onto.dice_0
* Owlready * Adding relation onto.dice_3 isAssociatedWith onto.dice_3
* Owlready * Adding relation onto.dice_3 isAssociatedWith onto.dice_5
* Owlready * Adding relation onto.dice_1 isAssociatedWith onto.dice_4
* Owlready * Adding relation onto.dice_1 isAssociatedWith onto.dice_0
* Owlready * Adding relation onto.dice_1 isAssociatedWith onto.dice_5
* Owlready * Adding 

* Owlready2 * Pellet took 4.5457305908203125 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)
